In [58]:
"""## TESTBENCH"""

# LeNet Architecture

# parameters
# conv1.weight (6, 1, 5, 5)
# conv2.weight (16, 6, 5, 5)
# fc1.weight (120,256)
# fc2.weight (84,120)
# fc3.weight (10,84)

# Input                 [1,28,28]
# Conv2d-1            [-1, 6, 24, 24]             150
# ReLU-2              [-1, 6, 24, 24]               0
# MaxPool2d-3         [-1, 6, 12, 12]               0
# Conv2d-4            [-1, 16, 8, 8]            2,400
# ReLU-5              [-1, 16, 8, 8]                0
# MaxPool2d-6         [-1, 16, 4, 4]                0
# Linear-7              [-1, 120]              30,720
# ReLU-8                [-1, 120]                   0
# Linear-9              [-1, 84]               10,080
# ReLU-10               [-1, 84]                    0
# Linear-11             [-1, 10]                  840

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

import math


import copy
import numpy as np




In [59]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST(root='../images/data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=16, pin_memory=True)

testset = datasets.MNIST(root='../images/data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=16, pin_memory=True)
dataset_sizes = {'train':len(trainset), 'test':len(testset)}


In [60]:
import torch.nn.functional as F
from  torch.ao.nn.quantized.functional import conv2d as qconv2d
from  torch.ao.nn.quantized.functional import linear as qlinear
def inference(inp_tensor,params):
  global temp1,temp2
  qin = torch.quantize_per_tensor(inp_tensor, params["quant.scale"].item(), params["quant.zero_point"].item(), torch.quint8)
  out = F.relu(qconv2d(input = qin, weight = params["conv1.weight"],bias = None,scale =params["conv1.scale"].item() ,zero_point =params["conv1.zero_point"].item() ))
  
  out = F.max_pool2d(input=out, kernel_size=2, stride=2)
  out = F.relu(qconv2d(input = out, weight = params["conv2.weight"],bias = None,scale =params["conv2.scale"].item() ,zero_point =params["conv2.zero_point"].item() ))
  out = F.max_pool2d(input=out, kernel_size=2, stride=2)
  

  out = out.reshape(out.shape[0], -1)
  
  # print(out.shape)
  out = F.relu(qlinear(input = out, weight = params["fc1._packed_params._packed_params"][0],bias = None,scale =params["fc1.scale"].item() ,zero_point =params["fc1.zero_point"].item() ))
  # temp1 = out
  out = F.relu(qlinear(input = out, weight = params["fc2._packed_params._packed_params"][0],bias = None,scale =params["fc2.scale"].item() ,zero_point =params["fc2.zero_point"].item() ))
  # temp2 = out
  out = qlinear(input = out, weight = params["fc3._packed_params._packed_params"][0],bias = None,scale =params["fc3.scale"].item() ,zero_point =params["fc3.zero_point"].item() )
  return out


In [61]:
num_correct = num_samples = 0

f = open("final_output.txt","r")

with torch.no_grad():
  for data in test_loader:
    inputs, labels  = data
    batch_size = inputs.shape[0]

    inputs = Variable(inputs.to("cpu"))
    labels = Variable(labels.to("cpu"))
    conv_data = []
    for i in range(10):
      conv_data.append(int(f.readline()))
    conv_tensor = torch.tensor(conv_data)
    _, preds = conv_tensor.max(0)
    num_correct += (preds == labels[0]).sum()
    print("Correct value is ",labels[0]," and the predicted value is ",preds)
    num_samples += 1
f.close()


Correct value is  tensor(7)  and the predicted value is  tensor(7)
Correct value is  tensor(2)  and the predicted value is  tensor(2)
Correct value is  tensor(1)  and the predicted value is  tensor(1)
Correct value is  tensor(0)  and the predicted value is  tensor(0)
Correct value is  tensor(4)  and the predicted value is  tensor(4)
Correct value is  tensor(1)  and the predicted value is  tensor(1)
Correct value is  tensor(4)  and the predicted value is  tensor(4)
Correct value is  tensor(9)  and the predicted value is  tensor(9)
Correct value is  tensor(5)  and the predicted value is  tensor(5)
Correct value is  tensor(9)  and the predicted value is  tensor(9)
Correct value is  tensor(0)  and the predicted value is  tensor(0)
Correct value is  tensor(6)  and the predicted value is  tensor(6)
Correct value is  tensor(9)  and the predicted value is  tensor(9)
Correct value is  tensor(0)  and the predicted value is  tensor(0)
Correct value is  tensor(1)  and the predicted value is  tenso

In [62]:
accuracy = (num_correct.item()/num_samples)*100
print("Model Predicted {} correctly out of {} from testing dataset, Acuracy : {:.2f}".format(num_correct.item(), num_samples, accuracy))


Model Predicted 9800 correctly out of 10000 from testing dataset, Acuracy : 98.00
